# Imports and library of basic functions

In this notebook, we provide the basic imports and self-defined functions needed for the implementation of the different concepts and results considered in our paper [\[1\]][1]. These will be used by the estimation strategies we propose (implemented in [estimation_strategies_with_examples.ipynb](estimation_strategies_with_examples.ipynb)) and for all the empirical results we report. Furthermore, we recap the several parametrizations we introduce in the [\[1\]][1] for the sawtooth signal and how to convert from one to the other.

## References

[\[1\]][1]: Pol del Aguila Pla, Lissy Pellaco, Satyam Dwivedi, Peter Händel and Joakim Jaldén, "Clock synchronization over networks: Identifiability of the sawtooth model", in IEEE Open Journal of Signal Processing, vol. 1, pp. 14-27, 2020, doi: 10.1109/OJSP.2020.2978762.

[1]:https://ieeexplore.ieee.org/document/9025036


## Index
<a id="index"></a>
1. [Import](#libs) libraries
2. Library of [basic functions](#basic)
    1. [Signal generation](#signal_generation) functions
    2. [Parameter transformation](#pars_transforms) functions
        1. From physical to standard parameters
        2. From standard to physical parameters
    3. [Interactive visualitzation](#eye_candy) functions

## Import libraries
<a id="libs"></a>
[Back to index](#index)

In [1]:
# Import useful packages
# Numpy for numerical computation
import numpy as np
# Itemgetter to make compressed .npz easier to retrieve
from operator import itemgetter
# Import numba for speeding up big numpy functions (not fully exploited)
import numba
from numba import jit
# Greatest common divisor from the math library (check co-primes)
from math import gcd
# Time to keep track of running times (simple, not a profiling tool)
import time
# Tabulate to make nice tables in examples
from tabulate import tabulate 

# Visualization libraries
# Matplotlib for plotting
import matplotlib.pyplot as plt
# Plotly for interactive plots (no need for go because we use iplot_mpl)
import plotly.offline as pltly
import plotly.tools as pltly_tls
# Connect to plotly
pltly.init_notebook_mode( connected = True )
# Matplotlib2tikz to easily generate tikz plots
from matplotlib2tikz import save as tikz_save

## Library of basic functions
<a id="basic"></a>
[Back to index](#index)

### Signal generation functions
<a id="signal_generation"></a>
[Back to index](#index)

We model the observed data as 

$$
    Y[n] = \alpha + \psi\, \mathrm{mod}_{1}\!\left( \beta n + \gamma + V[n] \right) + W[n]
$$

where $W[n]$ and $V[n]$ are independent white processes such that $V[n]\sim\mathcal{N}(0,\sigma_v^2)$ and $W[n]\sim\mathcal{N}(0,\sigma_w^2)$. 

In [2]:
# Our own version of mod_1(x). It avoids numerical errors in which mod_1(x) == 1.
@jit(nopython = True, parallel = True)
def mod_one( signal ):
        # Take Python's mod_1(x)
        signal = (signal)%(1)
        # Put all numbers within machine epsilon of 1 to 0 to comply with the mathematical definition
        signal = signal * np.logical_not( signal == 1 )
        
        return signal

In [3]:
# Generate a signal according to the sawtooth model
@jit(nopython = True, parallel = True)
def generate_signal( nrof_samples = 5000, beta = .001, gamma = np.random.rand(), alpha = 5e-6, psi = -1e-8, 
                    w_std = 0, v_std = 0):
    
    # Generate temporal axis
    n = np.linspace( 0, nrof_samples - 1, nrof_samples );
    # Generate a sawtooth signal with noise inside and outside
    y =  np.random.randn( nrof_samples ) * w_std  + alpha + psi * mod_one( 
         beta * n  + gamma + np.random.randn( nrof_samples ) * v_std ) 
    
    return y

### Parameter transformation functions
<a id="pars_transforms"></a>
[Back to index](#index)

A summary of the physical parameters

| Parameter | Meaning | Known to $\mathcal{M}$ | Known to $\mathcal{S}$ |
| -- | -- | -- | -- |
| $\delta_0$ | Delay introduced by $\mathcal{S}$, in seconds. | Yes. | Yes. |
| $\rho$ | Range between $\mathcal{S}$ and $\mathcal{M}$, in meters. | No. | No. | 
| $T_\mathcal{S}$ | Period of $\mathcal{S}$'s clock, in seconds. | No. | Yes. |
| $T_\mathrm{s} $ | Sampling period used by $\mathcal{M}$, in seconds. | Yes. | No. |
| $f_\mathrm{d}$ | Frequency difference between $\mathcal{S}$ and $\mathcal{M}$, in Hertz. | No. | No. |
| $\phi_\mathcal{S}$ | Phase parameter, in radians. | No. | No. |
| $\sigma_v$ | Standard deviation of the clock jitter (inside the modulus). | Yes. | Yes. |
| $\sigma_w$ | Standard deviation of other noise sources (outside the modulus). | Yes. | Yes. |


A summary of the standard parameters, all unknown to the observer of the signal.

| Parameter | Meaning   | Range of values         |
| --        | --        | --                      |
| $\alpha$  | Offset    | $\mathbb{R}$            |
| $\psi$    | Amplitude | $\mathbb{R}$            |
| $\beta$   | Frequency | $\left[-0.5,0.5\right)$ |
| $\gamma$  | Phase     | $[0,1)$                 |

In our estimation, we will assume that we have a relation between $\mathrm{h}(\cdot)$ such that given an estimate of $\beta$, $\hat{\beta}$, $\hat{\psi}=\mathrm{h}\left(\hat{\beta}\right)$ is a reasonable estimate of $\psi$. This function is implemented below in `psi_function`. The existance of this function parallels the case of joint clock synchronization and ranging.

#### From physical to standard parameters

Transformations, given between (10) and (11) in [\[1\]][1] 

[1]: https://arxiv.org

In [4]:
# Given some parameters for the physical clock synchronization problem, return the
# corresponding parameters for the standard sawtooth model
# See eq. between (10) and (11) in the paper
@jit(nopython = True)
def to_standard_parameters( T_sampling, delta_0, rho, f_d, T_s, phase_s ):
    # Required constants
    # Speed of light
    c = 3e8
    
    # Transformation of parameters
    # Offset from range, protocol delay and S's period
    alpha = delta_0 + 2 * rho / c + T_s
    # Obtain amplitude from T_s
    psi = - T_s
    # Obtain beta from sampling time and frequency difference
    beta = f_d * T_sampling
    # Phase delay from range and S's phase
    gamma = ( rho /( c * T_s ) + phase_s / (2 * np.pi) )%(1)
    
    # Return standard parameters
    return (alpha, beta, gamma, psi)

#### From standard to physical parameters

Transformations, given in (11) - (13) in [\[1\]][1]

[1]: https://arxiv.org

In [5]:
# Given some standard (estimated) sawtooth model parameters, and the parameters known
# to M, return the corresponding physical parameters for the clock synchronization problem
# See eq. (11) - (13) in the paper
@jit(nopython = True)
def to_physical_parameters( alpha, beta, gamma, T_m, T_sampling, delta_0 ):
    # Required constants
    # Speed of light
    c = 3e8
    
    # Transformation of parameters
    # Obtain frequency difference from beta and the sampling time
    f_d = beta / T_sampling
    
    # Extract other quantities that make the transformation easier
    # Obtain S's clock period from the frequency difference
    T_s = T_m / ( 1 + T_m * f_d )
    
    # Obtain the range
    rho = ( alpha - delta_0 - T_s ) * c / 2   
    # Obtain S's phase
    aux_1 = ( rho / ( c * T_s ) )%(1)
    aux_2 = ( gamma - aux_1 )%(1)
    phase_s = 2 * np.pi * aux_2
    # Return the physical clock synchronization and ranging parameters
    return (f_d, phase_s, rho)

 # Define a function for obtainig psi (-T_s) from beta and known values
@jit(nopython = True)
def psi_function( beta, T_sampling, T_m ):
    sampling_factor = T_sampling / T_m
    return (- T_m / (1 + beta / sampling_factor))

### Interactive visualization functions
<a id="eye_candy"></a>
[Back to index](#index)

In [6]:
def mpl_to_pltly( fig ):
    # Strip matplotlib figure of its legend (not supported by plotly)
    for child in fig.get_children( ):
        try:
            child.get_legend().remove()
        except:
            pass
    pltly_fig = pltly_tls.mpl_to_plotly( fig )
    pltly_fig['layout']['showlegend'] = True
    pltly.iplot( pltly_fig )